In [1]:
%%writefile seq.cu

#include <iostream>
#include <stdlib.h>
#include <sys/time.h>
#include <math.h>
#include <vector>
#include <string>

#define N 16 // Board size
#define DONE 65535  // 2^N - 1

// Retourner le temps
long long start_timer() {
	struct timeval tv;
	gettimeofday(&tv, NULL);
	return tv.tv_sec * 1000000 + tv.tv_usec;
}


// Retourner le temps écoulé depuis le temps spécifié spécifié
long long stop_timer(long long start_time, const char *name) {
	struct timeval tv;
	gettimeofday(&tv, NULL);
	long long end_time = tv.tv_sec * 1000000 + tv.tv_usec;
	printf("%s: %.5f sec\n", name, ((float) (end_time - start_time)) / (1000 * 1000));
	return end_time - start_time;
}

// Counts the number of non-zero bits in n
int countBits(long n) {
    int counter = 0;
    while (n) {
        n &= (n - 1); // This deletes the rightmost non-zero digit
        counter++;
    }
    return counter;
}

/*
Given the diagonals and columns attacked by the already placed queens in bit form, returns the amount of solutions.
So for the arguments 0, it finds all solutions, for 1, 2, 4 it finds all solutions with the first queen at the second
column, etc.
*/
long nQueens(long leftDiagonal, long column, long rightDiagonal) {

    long counter = 0;

    // When all columns are occupied, we have found a solution
    if (column == DONE) {
        return 1;
    }

    // the binary representation has a 1 on the places we can put a queen
    long possibilities = ~(leftDiagonal | column | rightDiagonal) & DONE; // & DONE to get rid of bits beyond N and fractions

    // stops when possibilities is 0, so when there are no possibilities anymore
    while (possibilities) {
        long bit = possibilities & (~possibilities + 1); // The rightmost non-zero bit in possibilities, the queen we place
        possibilities -= bit; // We have placed this queen, so remove it from the possibilities
        counter += nQueens((leftDiagonal | bit) >> 1, (column | bit), (rightDiagonal | bit) << 1); // We add the solution and shift the diagonals because the
                                                                                                    // queens attack diagonally one place further in the next row
    }

    return counter;
}

// Fills nodes with nodes in the k'th layer
long kLayer(std::vector<long>& nodes, int k, long leftDiagonal, long column, long rightDiagonal) {
    long counter = 0;

    // When all columns are occupied, we have found a solution
    if (countBits(column) == k) {
        nodes.push_back(leftDiagonal);
        nodes.push_back(column);
        nodes.push_back(rightDiagonal);
        return 1;
    }

    // the binary representation has a 1 on the places we can put a queen
    long possibilities = ~(leftDiagonal | column | rightDiagonal) & DONE; // & DONE to get rid of bits beyond N and fractions

    // stops when possibilities is 0, so when there are no possibilities anymore
    while (possibilities) {
        long bit = possibilities & (~possibilities + 1); // The rightmost non-zero bit in possibilities, the queen we place
        possibilities -= bit; // We have placed this queen, so remove it from the possibilities
        counter += kLayer(nodes, k, (leftDiagonal | bit) >> 1, (column | bit), (rightDiagonal | bit) << 1); // We add the solution and shift the diagonals because the
                                                                                                    // queens attack diagonally one place further in the next row
    }

    return counter;
}

// Returns a vector with all the nodes in the k'th layer
std::vector<long> kLayer(int k) {
    std::vector<long> nodes{};
    kLayer(nodes, k, 0, 0, 0);
    return nodes;
}

int main()
{
    // Vector of nodes (each encoded by 3 consecutive numbers) in the 3rd layer of the tree.
    std::vector<long> nodes = kLayer(4); // from layer 2 onwards we have an even amount of nodes in that layer, so we can use symmetry.
                                         // Layer 4 contains enough nodes (for N = 16, 9844).
    long long GPU_start_time = start_timer();

    // Calculate the solutions for each node
    long solutions = 0;
    for (int i = 0; i < nodes.size(); i += 3) {
        long leftDiagonal = nodes[i];
        long column = nodes[i + 1];
        long rightDiagonal = nodes[i + 2];
        solutions += nQueens(leftDiagonal, column, rightDiagonal);
    }

	long long GPU_time = stop_timer(GPU_start_time, "\tTotal execution time:");
    std::cout << "We have " << solutions << " solutions on a " << N << " by " << N << " board." << std::endl;

    return 0;
}


Writing seq.cu


In [2]:
%%shell
nvcc seq.cu -o seq -I./
./seq

	Total execution time:: 17.37384 sec
We have 14772512 solutions on a 16 by 16 board.


In [3]:
%%writefile par.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <sys/time.h>
#include <iostream>
#include <stdlib.h>
#include <math.h>
#include <vector>
#include <string>

#define N 16 // Board size
#define DONE  65535 // 2^N - 1

// Retourner le temps
long long start_timer() {
	struct timeval tv;
	gettimeofday(&tv, NULL);
	return tv.tv_sec * 1000000 + tv.tv_usec;
}


// Retourner le temps écoulé depuis le temps spécifié spécifié
long long stop_timer(long long start_time, const char *name) {
	struct timeval tv;
	gettimeofday(&tv, NULL);
	long long end_time = tv.tv_sec * 1000000 + tv.tv_usec;
	printf("%s: %.5f sec\n", name, ((float) (end_time - start_time)) / (1000 * 1000));
	return end_time - start_time;
}

// Counts the number of non-zero bits in n
int countBits(long n) {
    int counter = 0;
    while (n) {
        n &= (n - 1); // This deletes the rightmost non-zero digit
        counter++;
    }

    return counter;
}

/*
Given the diagonals and columns attacked by the already placed queens in bit form, returns the amount of solutions.
So for the arguments 0, it finds all solutions, for 1, 2, 4 it finds all solutions with the first queen at the second
column, etc.
*/
__host__ __device__ long nQueens(long leftDiagonal, long column, long rightDiagonal) {

    long counter = 0;

    // When all columns are occupied, we have found a solution
    if (column == DONE) {
        return 1;
    }

    // the binary representation has a 1 on the places we can put a queen
    long possibilities = ~(leftDiagonal | column | rightDiagonal) & DONE; // & DONE to get rid of bits beyond N and fractions

    // stops when possibilities is 0, so when there are no possibilities anymore
    while (possibilities) {
        long bit = possibilities & (~possibilities + 1); // The right most non-zero bit in possibilities, the queen we place
        possibilities -= bit; // We have placed this queen, so remove it from the possibilities
        counter += nQueens((leftDiagonal | bit) >> 1, (column | bit), (rightDiagonal | bit) << 1); // We add the solution and shift the diagonals because the
                                                                                               // queens attack diagonally one place further in the next row
    }

    return counter;
}

// Fills nodes with nodes in the k'th layer
long kLayer(std::vector<long>& nodes, int k, long leftDiagonal, long column, long rightDiagonal) {
    long counter = 0;

    // When all columns are occupied, we have found a solution
    if (countBits(column) == k) {
        nodes.push_back(leftDiagonal);
        nodes.push_back(column);
        nodes.push_back(rightDiagonal);
        return 1;
    }

    // the binary representation has a 1 on the places we can put a queen
    long possibilities = ~(leftDiagonal | column | rightDiagonal) & DONE; // & DONE to get rid of bits beyond N and fractions

    // stops when possibilities is 0, so when there are no possibilities anymore
    while (possibilities) {
        long bit = possibilities & (~possibilities + 1); // The right most non-zero bit in possibilities, the queen we place
        possibilities -= bit; // We have placed this queen, so remove it from the possibilities
        counter += kLayer(nodes, k, (leftDiagonal | bit) >> 1, (column | bit), (rightDiagonal | bit) << 1); // We add the solution and shift the diagonals because the
                                                                                               // queens attack diagonally one place further in the next row
    }

    return counter;
}

// Returns a vector with all the nodes in the k'th layer
std::vector<long> kLayer(int k) {
    std::vector<long> nodes{};
    kLayer(nodes, k, 0, 0, 0);
    return nodes;
}

// Takes an array of the nodes, an empty array solutions, where we fill the i'th member with the solutions
// in the i'th subtree.
__global__ void calculateSolutions(long* nodes, long* solutions, int numElements) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < numElements) {
        solutions[i] = nQueens(nodes[3 * i], nodes[3 * i + 1], nodes[3 * i + 2]);
    }
}


int main()
{
    // Vector of nodes (each encoded by 3 consecutive numbers) in the 3'rd layer of the tree.
    std::vector<long> nodes = kLayer(4); // from layer 2 onwards we have an even amount of nodes in that layer, so we can use symmetry.
                                         // Layer 4 contains enough nodes (for N = 16, 9844).

    // There are no classes on the device, so we point to the first element and then copy to the device.
    size_t nodeSize = nodes.size() * sizeof(long);
    long* hostNodes = (long*)malloc(nodeSize);
    hostNodes = &nodes[0];
    long* deviceNodes = NULL;
    long long GPU_start_time = start_timer();
    cudaMalloc((void**)&deviceNodes, nodeSize);
    cudaMemcpy(deviceNodes, hostNodes, nodeSize, cudaMemcpyHostToDevice);

    // Allocate the device output
    long* deviceSolutions = NULL;
    int numSolutions = nodes.size() / 6; // We only need half of the nodes, and each node is encoded by 3 integers.
    size_t solutionSize = numSolutions * sizeof(long);
    cudaMalloc((void**)&deviceSolutions, solutionSize);

    // Launch the nQueens CUDA Kernel
    int threadsPerBlock = 8;
    int blocksPerGrid = (numSolutions + threadsPerBlock - 1) / threadsPerBlock;
    calculateSolutions <<<blocksPerGrid, threadsPerBlock >>> (deviceNodes, deviceSolutions, numSolutions);

    // Copy results to host
    long* hostSolutions = (long*)malloc(solutionSize);
    cudaMemcpy(hostSolutions, deviceSolutions, solutionSize, cudaMemcpyDeviceToHost);

    // Add the subsolutions and print the result
    long solutions = 0;
    for (long i = 0; i < numSolutions; i++) {
        solutions += 2*hostSolutions[i]; // Symmetry
    }
	long long GPU_time = stop_timer(GPU_start_time, "\tTotal execution time:");

    std::cout << "We have " << solutions << " solutions on a " << N << " by " << N << " board." << std::endl;


    return 0;
}


Writing par.cu


In [4]:
%%shell
nvcc par.cu -o par -I./
./par

ptxas warning : Stack size for entry function '_Z18calculateSolutionsPlS_i' cannot be statically determined
	Total execution time:: 4.21771 sec
We have 14772512 solutions on a 16 by 16 board.


In [7]:
%%writefile cuda.cu

// N-Queens Placement and Solutions Counter
// Author: James Walker
// Copyrighted 2017 under the MIT license:
//   http://www.opensource.org/licenses/mit-license.php
//
// Purpose:
//   The N-Queens Counter follows an improved version of the algorithm used by
//   the N-Queens Solver, except it does not return any of the solutions.
//   Instead, the program counts the number of solutions for a given N-Queens
//   problem as well as the number of times a queen is placed during the
//   program's execution.
// Compilation, Execution, and Example Output:
//   $ gcc -std=c99 -O2 n_queens_counter.c -o n_queens_counter
//   $ ./n_queens_counter.exe 12
//   The 12-Queens problem required 428094 queen placements to find all 14200
//   solutions
//
// This implementation was adapted from the algorithm provided at the bottom of
// this webpage:
//   www.cs.utexas.edu/users/EWD/transcriptions/EWD03xx/EWD316.9.html

#include <inttypes.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <cuda_runtime.h>

#define MAX_N 32

// An abstract representation of an NxN chess board to tracking open positions
struct chess_board {
  uint32_t n_size;            // Number of queens on the NxN chess board
  uint32_t *queen_positions;  // Store queen positions on the board
  uint32_t *column;           // Store available column moves/attacks
  uint32_t *diagonal_up;      // Store available diagonal moves/attacks
  uint32_t *diagonal_down;
  uint32_t column_j;          // Stores column to place the next queen in
  uint64_t placements;        // Tracks total number queen placements
  uint64_t solutions;         // Tracks number of solutions
};
static struct chess_board *board;

// Handles dynamic memory allocation of the arrays and sets initial values
static void initialize_board(const uint32_t n_queens) {
  if (n_queens < 1) {
    fprintf(stderr, "The number of queens must be greater than 0.\n");
    exit(EXIT_SUCCESS);
  }

  // Dynamically allocate memory for chessboard struct
  board = (struct chess_board*)malloc(sizeof(struct chess_board));
  if (board == NULL) {
    fprintf(stderr, "Memory allocation failed for chess board.\n");
    exit(EXIT_FAILURE);
  }

  // Dynamically allocate memory for chessboard arrays that track positions
  const uint32_t diagonal_size = 2 * n_queens - 1;
  const uint32_t total_size = 2 * (n_queens + diagonal_size);
  board->queen_positions = (uint32_t*)malloc(sizeof(uint32_t) * total_size);
  if (board->queen_positions == NULL) {
    fprintf(stderr, "Memory allocation failed for the chess board arrays.\n");
    free(board);
    exit(EXIT_FAILURE);
  }
  board->column = &board->queen_positions[n_queens];
  board->diagonal_up = &board->column[n_queens];
  board->diagonal_down = &board->diagonal_up[diagonal_size];

  // Initialize the chess board parameters
  board->n_size = n_queens;
  for (uint32_t i = 0; i < n_queens; ++i) {
    board->queen_positions[i] = 0;
  }
  for (uint32_t i = n_queens; i < total_size; ++i) {
    // Initializes values for column, diagonal_up, and diagonal_down
    board->queen_positions[i] = 1;
  }
  board->column_j = 0;
  board->placements = 0;
  board->solutions = 0;
}

// Frees the dynamically allocated memory for the chess board structure
static void smash_board() {
  free(board->queen_positions);
  free(board);
}

__device__ uint32_t square_is_free(const uint32_t row_i, uint32_t* column, uint32_t* diagonal_up, uint32_t* diagonal_down, const uint32_t n_size, const uint32_t column_j) {
  return column[row_i] & diagonal_up[(n_size - 1) + (column_j - row_i)] & diagonal_down[column_j + row_i];
}

__device__ void set_queen(const uint32_t row_i, uint32_t* queen_positions, uint32_t* column, uint32_t* diagonal_up, uint32_t* diagonal_down, const uint32_t n_size, uint32_t column_j) {
  queen_positions[column_j] = row_i;
  column[row_i] = 0;
  diagonal_up[(n_size - 1) + (column_j - row_i)] = 0;
  diagonal_down[column_j + row_i] = 0;
  column_j++;
}

__device__ void remove_queen(const uint32_t row_i, uint32_t* queen_positions, uint32_t* column, uint32_t* diagonal_up, uint32_t* diagonal_down, const uint32_t column_j, const uint32_t n_size) {
  column_j--;
  diagonal_down[column_j + row_i] = 1;
  diagonal_up[(n_size - 1) + (column_j - row_i)] = 1;
  column[row_i] = 1;
}

__host__ __device__ void place_queen_kernel(uint32_t* queen_positions, uint32_t* column, uint32_t* diagonal_up, uint32_t* diagonal_down, const uint32_t n_size, const uint32_t row_boundary, uint32_t* solutions) {
  uint32_t column_j = blockIdx.x * blockDim.x + threadIdx.x;

  if (column_j < n_size) {
    uint32_t middle = column_j ? n_size : n_size >> 1;

    for (uint32_t row_i = 0; row_i < row_boundary; ++row_i) {
      if (square_is_free(row_i, column, diagonal_up, diagonal_down, n_size, column_j)) {
        set_queen(row_i, queen_positions, column, diagonal_up, diagonal_down, n_size, column_j);

        if (column_j == n_size - 1) {
          atomicAdd(solutions, 2);
        } else if (queen_positions[0] != middle) {
          place_queen_kernel(queen_positions, column, diagonal_up, diagonal_down, n_size, 0, solutions);
        } else {
          place_queen_kernel(queen_positions, column, diagonal_up, diagonal_down, n_size, 0, solutions);
        }

        remove_queen(row_i, queen_positions, column, diagonal_up, diagonal_down, column_j, n_size );
      }
    }
  }
}

__global__ void calculateSolutions(uint32_t* queen_positions, uint32_t* column, uint32_t* diagonal_up, uint32_t* diagonal_down, const uint32_t n_size, const uint32_t row_boundary, uint32_t* solutions) {
  uint32_t column_j = blockIdx.x * blockDim.x + threadIdx.x;

  if (column_j < n_size) {
    uint32_t middle = column_j ? n_size : n_size >> 1;

    for (uint32_t row_i = 0; row_i < row_boundary; ++row_i) {
      if (square_is_free(row_i, column, diagonal_up, diagonal_down, n_size, column_j)) {
        set_queen(row_i, queen_positions, column, diagonal_up, diagonal_down, n_size, column_j);

        if (column_j == n_size - 1) {
          atomicAdd(solutions, 2);
        } else if (queen_positions[0] != middle) {
          place_queen_kernel(queen_positions, column, diagonal_up, diagonal_down, n_size, 0, solutions);
        } else {
          place_queen_kernel(queen_positions, column, diagonal_up, diagonal_down, n_size, 0, solutions);
        }

        remove_queen(row_i, queen_positions, column, diagonal_up, diagonal_down, column_j, n_size );
      }
    }
  }
}

int main(int argc, char* argv[]) {
  const uint32_t default_n = 4;
  const uint32_t n_queens = (argc != 1) ? (uint32_t)atoi(argv[1]) : default_n;

  const uint32_t row_boundary = (n_queens >> 1) + (n_queens & 1);
  const uint32_t total_size = 2 * (n_queens + (2 * n_queens - 1));
  uint32_t* queen_positions;
  uint32_t* column;
  uint32_t* diagonal_up;
  uint32_t* diagonal_down;
  uint32_t* solutions;

  cudaMallocManaged(&queen_positions, n_queens * sizeof(uint32_t));
  cudaMallocManaged(&column, n_queens * sizeof(uint32_t));
  cudaMallocManaged(&diagonal_up, total_size * sizeof(uint32_t));
  cudaMallocManaged(&diagonal_down, total_size * sizeof(uint32_t));
  cudaMallocManaged(&solutions, sizeof(uint32_t));

  cudaMemset(queen_positions, 0, n_queens * sizeof(uint32_t));
  cudaMemset(column, 1, n_queens * sizeof(uint32_t));
  cudaMemset(diagonal_up, 1, total_size * sizeof(uint32_t));
  cudaMemset(diagonal_down, 1, total_size * sizeof(uint32_t));
  cudaMemset(solutions, 0, sizeof(uint32_t));

  calculateSolutions<<<1, n_queens>>>(queen_positions, column, diagonal_up, diagonal_down, n_queens, row_boundary, solutions);
  cudaDeviceSynchronize();

  uint32_t solution_count;
  cudaMemcpy(&solution_count, solutions, sizeof(uint32_t), cudaMemcpyDeviceToHost);
  printf("Number of solutions for %d queens: %d\n", n_queens, solution_count);

  cudaFree(queen_positions);
  cudaFree(column);
  cudaFree(diagonal_up);
  cudaFree(diagonal_down);
  cudaFree(solutions);

  return 0;
}





Overwriting cuda.cu


In [9]:
%%shell
nvcc cuda.cu -o n_queens_counter
./n_queens_counter 12

cuda.cu(76): error: expression must be a modifiable lvalue

cuda.cu(22): warning #177-D: function "initialize_board" was declared but never referenced

cuda.cu(58): warning #177-D: function "smash_board" was declared but never referenced

1 error detected in the compilation of "cuda.cu".
/bin/bash: line 1: ./n_queens_counter: No such file or directory


CalledProcessError: ignored